# stocks 정보 최신화

In [1]:
# pykrx to mysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from openpyxl import load_workbook
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

start = '20190101'
end = '20211018'
# end = str(datetime.now().date()).replace('-', '')
period = pd.date_range(start=start, end=end).strftime("%Y%m%d").tolist()

for date in period:
  try:
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
    conn = db_connection.connect()
    df = stock.get_market_ohlcv_by_ticker(date, market="KOSPI")
    time.sleep(0.5)
    if (df['시가'][0] == 0):
      continue
    df['date'] = date
    df = df.reset_index().set_index('date').drop(['거래대금', '등락률'], axis=1).rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, date)
    pass


# 볼린저밴드 구하기

In [30]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      # print(ticker[0], date[0].strftime("%Y%m%d"))
      try:
        getLast20InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit 20"
        cursor.execute(getLast20InfoSql)
        last20InfoDf = pd.DataFrame(cursor.fetchall())
        last20InfoDfClose = last20InfoDf.loc[:, 5]

        avg = last20InfoDfClose.mean()
        std = last20InfoDfClose.std()
        tmpInfo = last20InfoDf.loc[0, [0, 1, 5]]
        hi = round(avg + std * 2)
        me = round(avg)
        lo = round(avg - std * 2)
        bw = (hi - lo) / me
        insertBollSql = "REPLACE INTO boll(date, ticker, close, low, medium, high, bandWidth) VALUES(%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw))
        db.commit()
      except Exception as ex1:
        print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
        # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

ex1 cannot convert float NaN to integer 000020 20190102
ex1 cannot convert float NaN to integer 000030 20190102
ex1 cannot convert float NaN to integer 000040 20190102
ex1 cannot convert float NaN to integer 000050 20190102
ex1 cannot convert float NaN to integer 000060 20190102
ex1 cannot convert float NaN to integer 000070 20190102
ex1 cannot convert float NaN to integer 000075 20190102
ex1 cannot convert float NaN to integer 000080 20190102
ex1 cannot convert float NaN to integer 000087 20190102
ex1 cannot convert float NaN to integer 000100 20190102
ex1 cannot convert float NaN to integer 000105 20190102
ex1 cannot convert float NaN to integer 000120 20190102
ex1 cannot convert float NaN to integer 000140 20190102
ex1 cannot convert float NaN to integer 000145 20190102
ex1 cannot convert float NaN to integer 000150 20190102
ex1 cannot convert float NaN to integer 000155 20190102
ex1 cannot convert float NaN to integer 000157 20190102
ex1 cannot convert float NaN to integer 000180 2

In [ ]:

# last20InfoDf.loc[:, [0, 1, 2, 3]]
last20InfoDf
# last20InfoDfClose = last20InfoDf.loc[:, 3]
# tmpInfo[5]
# avg = last20InfoDfClose.mean()
# std = last20InfoDfClose.std()

# # print(type(avg))
# hi = round(avg + std * 2)
# mi = round(avg)
# lo = round(avg - std * 2)
# print(hi, mi, lo)
# last20InfoDf.loc[0, [0, 1, 2, 3]]

17700

# Z-SCORE
- (현재값 - 평균) / 표준편차

In [ ]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10, 20, 30, 90, 180]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        endDate = date[0].strftime("%Y%m%d")

        try:
          getLastNBollSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day)
          cursor.execute(getLast20InfoSql)
          lastNBollDf = pd.DataFrame(cursor.fetchall())

          if len(lastNBollDf.index) < day:
            continue
          lastNBollBandWidthDf = lastNBollDf.loc[:, 6]

          avg = lastNBollBandWidthDf.mean()
          std = lastNBollBandWidthDf.std()
          tmpInfo = lastNBollDf.loc[0, [0, 1, 6]]
          z_score = (tmpInfo[6] - avg) / std
          
          insertBollSql = "REPLACE INTO boll(date, ticker, period, bandWidth, z_score) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], day, tmpInfo[6], z_score))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()